In [1]:
!nvidia-smi

Fri Nov  4 11:16:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.154                Driver Version: 390.154                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:73:00.0  On |                  N/A |
| 42%   70C    P2   109W / 250W |    706MiB / 11169MiB |     78%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Import libraries 

In [4]:
import gromacs 
import os

In [5]:
os.chdir('/home/ali/cg_simulations/new_simu1')

# Prepare the system

In [ ]:
!martinize2 -f model_start_aligned.pdb -ff martini3001 -x start_cg.pdb -o topoltemp.top -dssp /usr/bin/mkdssp -elastic -scfix -cys 0.40 -p backbone -pf 1000

In [ ]:
!python2.7 insane_altered.py -salt 0.15 -charge -0 -sol W -f start_cg.pdb -o system.gro -pbc rectangular -d 30 -x 9 -y 9 -z 12 -l POPC:100 -p topol.top

In [ ]:
!sed -i -e "s/Protein        1/molecule_0 1\nmolecule_1 1/g" ./topol.top

In [ ]:
!cat topol.top

# Energy Minimization

In [ ]:
#######6.0 double précision, pas trop utile#########

#!gmx grompp -f step6.0_minimization.mdp -o step6.0_minimization.tpr -c system.gro -r system.gro -p topol.top
!gmx_mpi grompp -f step6.0_minimization.mdp -o step6.0_minimization.tpr -c system.gro -r system.gro -p topol.top

In [ ]:
#!gmx mdrun -deffnm step6.0_minimization -gpu_id 0
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.0_minimization -ntomp 5 -v

In [ ]:
#!gmx grompp -f step6.1_minimization.mdp -o step6.1_minimization.tpr -c step6.0_minimization.gro -r step6.0_minimization.gro -p topol.top
!gmx_mpi grompp -f step6.1_minimization.mdp -o step6.1_minimization.tpr -c step6.0_minimization.gro -r step6.0_minimization.gro -p topol.top

In [ ]:
#!gmx mdrun -rdd 3.0 -deffnm step6.1_minimization -v
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.1_minimization -ntomp 5 -v

# Equilibration Steps - NVT

In [ ]:
#!gmx grompp -f step6.2_equilibration.mdp -o step6.2_equilibration.tpr -c step6.1_minimization.gro -r step6.1_minimization.gro -p topol.top
!gmx_mpi grompp -f step6.2_equilibration.mdp -o step6.2_equilibration.tpr -c step6.1_minimization.gro -r step6.1_minimization.gro -p topol.top

In [ ]:
#!gmx mdrun -rdd 3.0 -deffnm step6.2_equilibration -gpu_id 0 -v -nt 18
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.2_equilibration -ntomp 5 -gpu_id 0 -v

# Equilibration Steps - NPT

In [ ]:
#!gmx grompp -f step6.3_equilibration.mdp -o step6.3_equilibration.tpr -c step6.2_equilibration.gro -r step6.2_equilibration.gro -p topol.top
!gmx_mpi grompp -f step6.3_equilibration.mdp -o step6.3_equilibration.tpr -c step6.2_equilibration.gro -r step6.2_equilibration.gro -p topol.top

In [ ]:
#!gmx mdrun -rdd 3.0 -deffnm step6.3_equilibration -gpu_id 0 -v -nt 18
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.3_equilibration -ntomp 5 -gpu_id 0 -v

In [ ]:
#!gmx grompp -f step6.4_equilibration.mdp -o step6.4_equilibration.tpr -c step6.3_equilibration.gro -r step6.3_equilibration.gro -p topol.top
!gmx_mpi grompp -f step6.4_equilibration.mdp -o step6.4_equilibration.tpr -c step6.3_equilibration.gro -r step6.3_equilibration.gro -p topol.top

In [ ]:
#!gmx mdrun -rdd 3.0 -deffnm step6.4_equilibration -gpu_id 0 -v -nt 18
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.4_equilibration -ntomp 5 -gpu_id 0 -v

In [ ]:
#!gmx grompp -f step6.5_equilibration.mdp -o step6.5_equilibration.tpr -c step6.4_equilibration.gro -r step6.4_equilibration.gro -p topol.top
!gmx_mpi grompp -f step6.5_equilibration.mdp -o step6.5_equilibration.tpr -c step6.4_equilibration.gro -r step6.4_equilibration.gro -p topol.top

In [ ]:
#!gmx mdrun -rdd 3.0 -deffnm step6.5_equilibration -gpu_id 0 -v -nt 18
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.5_equilibration -ntomp 5 -gpu_id 0 -v

In [ ]:
#!gmx grompp -f step6.6_equilibration.mdp -o step6.6_equilibration.tpr -c step6.5_equilibration.gro -r step6.5_equilibration.gro -p topol.top
!gmx_mpi grompp -f step6.6_equilibration.mdp -o step6.6_equilibration.tpr -c step6.5_equilibration.gro -r step6.5_equilibration.gro -p topol.top

In [ ]:
#!gmx mdrun -rdd 3.0 -deffnm step6.6_equilibration -gpu_id 0 -v -nt 18
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.6_equilibration -ntomp 5 -gpu_id 0 -v

# MD run 

In [ ]:
#!gmx grompp -f step7_production.mdp -o step7_production.tpr -c step6.6_equilibration.gro -r step6.6_equilibration.gro -p topol.top
!gmx_mpi grompp -f step7_production.mdp -o step7_production.tpr -c step6.6_equilibration.gro -r step6.6_equilibration.gro -p topol.top

In [ ]:
#!gmx mdrun -rdd 3.0 -deffnm step7_production -gpu_id 0 -v -nt 18
!gmx_mpi mdrun -rdd 3.0 -deffnm step7_production -ntomp 5 -gpu_id 0 -v

In [ ]:
!gmx check -f step7_production.xtc

In [ ]:
#!gmx mdrun -rdd 3.0 -deffnm step7_production -cpi step7_production.cpt -gpu_id 0 -v -nt 18

# make an Index

In [6]:
#get the number of the first and last atoms of the receptor and peptide
!cat system.gro

Protein in INSANE! Membrane UpperLeaflet>POPC=100.0 LowerLeaflet>POPC=100.0
 8686
   40ALA   BB      1   5.159   2.245   7.809
   40ALA   SC1     2   5.265   2.108   7.933
   41PRO   BB      3   5.057   2.210   7.524
   41PRO   SC1     4   5.009   1.983   7.640
   42LEU   BB      5   4.957   2.490   7.488
   42LEU   SC1     6   4.658   2.435   7.552
   43LEU   BB      7   5.211   2.637   7.539
   43LEU   SC1     8   5.219   2.716   7.859
   44ALA   BB      9   5.376   2.522   7.319
   44ALA   SC1    10   5.444   2.327   7.373
   45GLY   BB     11   5.197   2.599   7.087
   46VAL   BB     12   5.182   2.899   7.139
   46VAL   SC1    13   5.053   2.974   7.317
   47THR   BB     14   5.477   2.953   7.133
   47THR   SC1    15   5.581   2.909   7.364
   48ALA   BB     16   5.530   2.859   6.851
   48ALA   SC1    17   5.538   2.646   6.849
   49THR   BB     18   5.340   3.044   6.710
   49THR   SC1    19   5.108   3.013   6.716
   50CYS   BB     20   5.456   3.308   6.807
   50CYS   SC1    

In [8]:
######################### In the terminal: (could'nt make it work here)######################################

!echo -e "a 1-725  \nname 22 rec \na 726-750 \nname 23 pep \nq" |gmx make_ndx -f system.gro -o index.ndx

#a 1-725
# name 22 rec
# a 726-750
# name 23 pep

                     :-) GROMACS - gmx make_ndx, 2020.4 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
    

# Center the the box

In [9]:
!echo 22 0|gmx trjconv -f step7_production.xtc -o step7_production.pbc.xtc -s step7_production.tpr -pbc mol -center -n index.ndx 

                     :-) GROMACS - gmx trjconv, 2020.4 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

# NOW READY TO BE ANALYZED!!